In [5]:
import numpy as np

from keras.models import Sequential
from keras.layers import Embedding, Dense
from keras.layers.recurrent import LSTM

from data.models import SentenceBatch, Task
from keras_ml.models import Model, TrainSession
from word_embeddings.utils import load_w2v_model
from keras import backend as K

from tqdm import tqdm_notebook
import itertools

In [6]:
task = Task.objects.get(
    type=Task.Type.ASPECT_DETECTION,
    aspect_entity='AMBIENCE',
    aspect_attribute='GENERAL'
)
batch = SentenceBatch.objects.filter(task=task).first()
w2v_model = load_w2v_model(batch.w2v_model)

Loading w2v model using cache


In [7]:
K.clear_session()
embedding_layer = Embedding(
    input_dim=len(w2v_model.wv.vocab),
    output_dim=w2v_model.vector_size,
    input_length=len(batch.x_train[0]),
    mask_zero=True
)

lstm_layer = LSTM(
    units=200,
    activation='selu',
    recurrent_activation='hard_sigmoid'
)
dense_layer = Dense(1, activation='sigmoid')

model = Sequential()
model.add(embedding_layer)
model.add(lstm_layer)
model.add(dense_layer)

config = model.get_config()
compile_opts = dict(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
train_opts = dict(
    epochs=10,
    validation_split=0.1,
    batch_size=128
)

model.compile(**compile_opts)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 60, 300)           9687300   
_________________________________________________________________
lstm_1 (LSTM)                (None, 200)               400800    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 10,088,301
Trainable params: 10,088,301
Non-trainable params: 0
_________________________________________________________________


In [ ]:
config

In [ ]:
model.get_config()

In [ ]:
x_train = np.array(batch.x_train)
y_train = np.array(batch.y_train)
# model.fit(x_train, y_train, **train_opts)

In [9]:
keras_model, _ = Model.objects.get_or_create(
    config=model.get_config(),
    compile_opts=compile_opts
)

In [ ]:
TrainSession.objects.create(
            model=keras_model,
            batch=batch,
            train_opts=train_opts
        )

In [ ]:
TrainSession.objects.count()

In [ ]:
runs = 1
SentenceBatch.objects.filter(task__type=Task.Type.ASPECT_DETECTION).count() * runs * 100

In [ ]:
for batch in tqdm_notebook(SentenceBatch.objects.filter(task__type=Task.Type.ASPECT_DETECTION)):
    for _ in range(runs):
        TrainSession.objects.create(
            model=keras_model,
            batch=batch,
            train_opts=train_opts
        )

In [ ]:
Model.objects.count()

In [ ]:
activations = ['softmax', 'elu', 'selu', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
activation_pairs = []
for activation, recurrent_activation in tqdm_notebook(itertools.product(activations, repeat=2)):
    K.clear_session()
    # Create model
    embedding_layer = Embedding(
    input_dim=len(w2v_model.wv.vocab),
    output_dim=w2v_model.vector_size,
    input_length=len(batch.x_train[0]),
    mask_zero=True
    )

    lstm_layer = LSTM(
        units=200,
        activation=activation,
        recurrent_activation=recurrent_activation
    )
    dense_layer = Dense(1, activation='sigmoid')

    model = Sequential()
    model.add(embedding_layer)
    model.add(lstm_layer)
    model.add(dense_layer)

    config = model.get_config()
    compile_opts = dict(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    train_opts = dict(
        epochs=10,
        validation_split=0.1,
        batch_size=128
    )
    
    keras_model, _ = Model.objects.get_or_create(
        config=model.get_config(),
        compile_opts=compile_opts
    )


In [ ]:
for model in tqdm_notebook(Model.objects.all()):
    for batch in SentenceBatch.objects.filter(task__type=Task.Type.ASPECT_DETECTION):
        TrainSession.objects.create(
            model=model,
            batch=batch,
            train_opts=train_opts
        )

In [ ]:
# POLARITY DETECTION

In [10]:
for batch in tqdm_notebook(SentenceBatch.objects.filter(task__type=Task.Type.POLARITY_DETECTION)):
    for _ in range(1):
        TrainSession.objects.create(
            model=keras_model,
            batch=batch,
            train_opts=train_opts
        )

HBox(children=(IntProgress(value=0, max=3), HTML(value='')))